In [1]:
import os 
import torch 
import esm
import json 
from tqdm import tqdm 
import numpy as np
from scipy.special import softmax
from utils import *
import random 
import pickle
import matplotlib.pyplot as plt
import random

In [2]:
with open('../data/full_seq_dict.json', "r") as json_file:
    seq_dict = json.load(json_file)

In [3]:
with open('../data/shuffled_selected_protein.json', "r") as json_file:
    shuffled = json.load(json_file)

In [4]:
shuffled[:10]

['2AMHA',
 '2X7QA',
 '4J6OA',
 '4RD4A',
 '1RKQA',
 '2DWUA',
 '4YJ6A',
 '1TXGA',
 '4NN3A',
 '3ATSA']

In [5]:
os.environ["CUDA_VISIBLE_DEVICES"] = '3'

In [6]:
model, alphabet = esm.pretrained.esm2_t6_8M_UR50D()

In [7]:
# put model on GPU if available
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
model = model.to(device)
model = model.eval()
#model.args.token_dropout = False

esm_alphabet = "".join(alphabet.all_toks[4:24])+"-"

In [8]:
def get_logits(seq):
  x,ln = alphabet.get_batch_converter()([("seq",seq)])[-1],len(seq)
  with torch.no_grad():
    f = lambda x: model(x)["logits"][0,1:(ln+1),4:24].cpu().numpy()
    x = x.to(device)
    logits = f(x)
    return logits

def get_masked_logits(seq, p=None, get_pll=False):
  x,ln = alphabet.get_batch_converter()([(None,seq)])[-1],len(seq)
  if p is None: p = ln
  with torch.no_grad():
    def f(x):
      fx = model(x)["logits"][:,1:(ln+1),4:24]
      return fx

    logits = np.zeros((ln,20))
    for n in range(0,ln,p):
      m = min(n+p,ln)
      x_h = torch.tile(torch.clone(x),[m-n,1])
      for i in range(m-n):
        x_h[i,n+i+1] = alphabet.mask_idx
      fx_h = f(x_h.to(device))
      for i in range(m-n):
        logits[n+i] = fx_h[i,n+i].cpu().numpy()
  if get_pll:
    logits = np.log(softmax(logits,-1))
    x = x.cpu().numpy()[0]
    x = x[1:(ln+1)] - 4
    return sum([logits[n,i] for n,i in enumerate(x)])
  else:
    return logits

In [9]:
def get_categorical_jacobian(seq):
  # ∂in/∂out
  x,ln = alphabet.get_batch_converter()([("seq",seq)])[-1],len(seq)
  with torch.no_grad():
    f = lambda x: model(x)["logits"][...,1:(ln+1),4:24].cpu().numpy()
    fx = f(x.to(device))[0]
    x = torch.tile(x,[20,1]).to(device)
    fx_h = np.zeros((ln,20,ln,20))
    for n in range(ln): # for each position
      x_h = torch.clone(x)
      x_h[:,n+1] = torch.arange(4,24) # mutate to all 20 aa
      fx_h[n] = f(x_h)
    return fx-fx_h

In [10]:
def get_data(pdb): 
    seq = seq_dict[pdb]
    # jacobian of the model
    jac = get_categorical_jacobian(seq)
    # center & symmetrize
    for i in range(4): jac -= jac.mean(i,keepdims=True)
    jac = (jac + jac.transpose(2,3,0,1))/2
    jac_contacts = get_contacts(jac)
    #print(jac.shape)
    
    np.save('../results/jac/8M/' + pdb + '_esm2_jac.npy', jac.astype(np.float16))
    np.save('../results/jac_contact/8M/' + pdb + '_esm2_jac_contact.npy', jac_contacts.astype(np.float16))

In [ ]:
for pdb in tqdm(shuffled):
    print(pdb)
    get_data(pdb)

  0%|          | 0/1431 [00:00<?, ?it/s]

2AMHA


  0%|          | 1/1431 [00:03<1:13:34,  3.09s/it]

2X7QA


  0%|          | 2/1431 [00:10<2:11:37,  5.53s/it]

...

4APXB


 67%|██████▋   | 961/1431 [2:15:53<1:16:29,  9.76s/it]

3MB5A


 67%|██████▋   | 962/1431 [2:15:57<1:04:04,  8.20s/it]

4DOYA
